In [17]:
from data_preprocessing.DataCleaner import DataCleaner
from data_preprocessing.DataLoader import DataLoader
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import numpy as np
import category_encoders as ce
import re
from scipy import stats
import pgeocode
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [18]:
df = DataLoader(path="car_data.parquet.gzip").load_parquet()
data_cleaner = DataCleaner(df)
df['price'] = [data_cleaner.clean_price(x) for x in df.price]

# Columns to modify pre-split
These columns are not learning any relationship from the data (they are not ordered). These columns are predominantly boolean flags or simple -1, 0, 1 encoding. Also calculate vehicle age here

In [19]:
"""
The below columns are simple boolean flags or simple encodings
"""

# Add drops to these funcs?
def add_service_history_flag():
    df['has_service_history'] = [1 if x is not None else 0 for x in df.service_history] 

def add_imported_flag():
    df['is_imported'] = [1 if x else 0 for x in df.imported]

def add_website_flag():
    df['has_website'] = [1 if x is not None else 0 for x in df.sellerwebsite]

def add_trim_flag():
    df['has_trim'] = [1 if x is not None else 0 for x in df.trim]

def add_ulez_flag():
    df['is_ulez'] = [1 if x == 'ULEZ' else 0 for x in df.emission_scheme]

def add_convertible_flag():
    df['is_convertible'] = [1 if x == 'Convertible' else 0 for x in df.body_type]

def add_vrm_flag():
    df['known_reg_plate'] = [1 if x else 0 for x in df.vrm]

def extract_manufactured_year(x):
    if isinstance(x, str):
        return int(x[:4])
    return x

def add_manufactured_year():
    df['year_of_manufacture'] = [extract_manufactured_year(x) for x in df.manufactured_year]

def _reg_checker(reg_no):
    if isinstance(reg_no, str):
        # Returns True if plat matches modern standards (from 2001 September)
        pattern = re.compile(r'^[A-Z]{2}[0-9]{2}[A-Z]{3}$')
        reg_return = pattern.findall(reg_no)
        if reg_return:
            return True
        return False
    return False

def is_private_plate(reg, year):
    if year >= 2002: # new systen
        if not _reg_checker(reg):
            return 1
        return 0
    else:
        return 0

def add_private_plate_flag():
    try:
        df['is_private_plate'] = [is_private_plate(reg, year) for reg, year in zip(df.vrm, df.year_of_manufacture)]
    except Exception:
        print("year of manufacture column not created yet")    

def extract_valid_reg_code(x):
    """
    CURRENTLY UNUSED
    Need to handle nulls and need to handle really old cars that do not have a (## reg) pattern
    Need to impute some reg codes too. Some newer vehicles dont have reg code
    """
    # ^.*?\([^\d]*(\d+)[^\d]*\).*$
    if isinstance(x, str):
        pattern = re.compile(r'\(([^\)]+)\)')

        x_return = pattern.findall(x)
        if x_return:
            return x_return[0].replace(' reg', '')
    if x is None:
        print(x)
    else:
        return x

def calculate_vehicle_age():
    df['year_of_scrape'] = [int(x[-4:]) for x in df.todaysdate]
    df['vehicle_age'] = df['year_of_scrape'] - df['year_of_manufacture']
    df.drop(columns=['year_of_scrape'], inplace=True)

def encode_mileage_deviation(x):
    if isinstance(x, str):
        if 'ABOVE' in x:
            return 1
        elif 'EQUAL' in x:
            return 0
        elif 'BELOW' in x:
            return -1
    return 0

def group_on_quartile(x, column, df=df):
    """
    UTIL
    """
    try:
        percentile = stats.percentileofscore(df[column], x, kind='rank')
    except Exception:
        return 0
    if percentile < 25:
        return 1
    elif 25 <= percentile < 50:
        return 2
    elif 50 <= percentile < 75:
        return 3
    else:
        return 4

def add_mileage_deviation():
    df['mileage_deviation_encoded'] = [encode_mileage_deviation(x) for x in df.mileageDeviation]

def find_rare(col, threshold=0.01):
    """
    Find the uncommon labels in a set and group them as 'rare'
    """
    vcs = col.value_counts().astype(int) # This is behaving weirdly
    n = len(col)
    rare_labels = []
    for k in vcs.keys():
        vcs[k] /= n
        if vcs[k] <= threshold:
            rare_labels.append(k)
    return rare_labels

def remove_whitespace_and_symbols(x):
    x = str(x).lower()
    x = re.sub(r'\W', '', x) # Symbols + whitespace
    return x

def group_care_makes():
    rare_labels = find_rare(df.make)
    df['make_grouped'] = ['rare' if x in rare_labels else remove_whitespace_and_symbols(x) for x in df['make']]

def encode_with_hashing(df, x, n_components=8):
    local_copy = df.copy()
    hash_encoder = ce.HashingEncoder(n_components=n_components, hash_method='sha256', cols=[x])
    dfbin = hash_encoder.fit_transform(local_copy[x])
    for col in dfbin.columns:
        dfbin.rename(columns={col: f"{x}_{col}"}, inplace=True)
    local_copy = pd.concat([df, dfbin], axis=1)
    local_copy.drop(columns=[x], inplace=True)
    return local_copy
    
    
def add_manual_flag():
    df['is_manual'] = [0 if x == 'Automatic' else 1 for x in df.transmission]


def group_fuel_types():
    fuel_rare_labels = find_rare(df.fuel_type)
    to_exclude = ["Hybrid – Diesel/Electric", "Hybrid – Diesel/Electric Plug-in"]
    for exclude in to_exclude:
        fuel_rare_labels.remove(exclude)

    def _remove_plugin_from_fuel(x):
        x = re.sub(r'plugin', '', x)
        return x
        
    df['fuel_type_grouped'] = ['other' if x in fuel_rare_labels else _remove_plugin_from_fuel(remove_whitespace_and_symbols(x)) for x in df.fuel_type]


def group_doors():
    df['doors'].fillna('unknown', inplace=True)
    door_rare_labels = find_rare(df.doors)
    df['doors_grouped'] = ['>5 doors' if x in door_rare_labels and x != 'unknown' else x for x in df.doors]


def group_seats():
    df['seats'].fillna('unknown', inplace=True)
    seat_rare_labels = find_rare(df.seats)
    df['seats_grouped'] = ['>6 seats' if x in seat_rare_labels and x != 'unknown' else x for x in df.seats]

def extract_integer(pattern, x):
    if isinstance(x, str):
        x = re.sub(pattern, '', x)
        x = x.replace(',', '')
        return int(x)
    return x

def mileage_to_integer():

    df['mileage'] = [extract_integer(r'mile[s]?', x) for x in df['mileage']]



def group_by_num_owners(x):
    try:
        x = int(x) # np.nan can't be converted to int
    except ValueError:
        return 'unknown'
    if x == 1:
        return '1 owner'
    elif 2 <= x <= 4:
        return '2to4 owners'
    elif 5 <= x <= 9:
        return '5to9 owners'
    else:
        return '10+ owners'


def group_owners():
    df['owners'] = [extract_integer(r'owner[s]?', x) for x in df['owners']]
    df['owners_grouped'] = [group_by_num_owners(x) for x in df['owners']]


def group_engine_sizes():
    def engine_to_numerical(x):
        try:
            x = x[:-1]
            return float(x)
        except:
            return x

    def group_engine_size(x):
        if x <= 1.0:
            return 'small'
        elif 1.0 < x < 3.0:
            return 'medium'
        elif 3.0 <= x < 15.0:
            return 'large'
        elif 15.0 < x < 500: # Assume error in input
            x /= 10
            return group_engine_size(x)
        elif x >= 500: # Assume engine is in CC
            x /= 1000
            return group_engine_size(x)
        else:
            return 'unknown'

    df['engine_size'] = [engine_to_numerical(x) for x in df.engine_size]
    df['engine_size_grouped'] = [group_engine_size(x) for x in df.engine_size]

    
def add_new_flag():
    df['is_new'] = [1 if x == 'New' else 0 for x in df.condition]


def co2_to_numerical(x):
    if isinstance(x, str):
        return int(x[:-4])
    else:
        return x

def co2_grouped(x):
    x = co2_to_numerical(x)
    if x == np.nan:
        return 'unknown'
    elif x == 0:
        return 'band1'
    elif 1 <= x <= 50:
        return 'band2'
    elif 51 <= x <= 75:
        return 'band3'
    elif 76 <= x <= 90:
        return 'band4'
    elif 91 <= x <= 100:
        return 'band5'
    elif 101 <= x <= 110:
        return 'band6'
    elif 111 <= x <= 130:
        return 'band7'
    elif 131 <= x <= 150:
        return 'band8'
    elif 151 <= x <= 170:
        return 'band9'
    elif 171 <= x <= 190:
        return 'band10'
    elif 191 <= x <= 225:
        return 'band11'
    elif 226 <= x <= 255:
        return 'band12'
    else:
        return 'band13'

def group_co2():
    df['co2_grouped'] = [co2_grouped(x) for x in df.co2Emissions]


def count_images():
    df['image_count'] = [x.count('https') if isinstance(x, str) else 0 for x in df.images]


def add_postcode_flag():
    df['has_postcode'] = [1 if x else 0 for x in df.sellerpostcode]


def single_pt_haversine(lat, lng, decimal_degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if decimal_degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d

def extract_lat_long_from_postcode():
    nomi = pgeocode.Nominatim('gb')
    postcodes = list(df.sellerpostcode)
    geo_df = nomi.query_postal_code(postcodes)

    df['long_lat_feature'] = [single_pt_haversine(lat, long, decimal_degrees=True) for long, lat in zip(geo_df.longitude, geo_df.latitude)]


def group_annual_tax():
    df['annual_tax_grouped'] = [group_on_quartile(x, 'annual_tax') for x in df.annual_tax]


def get_sentiment(text, sid_obj, raw=False):
    """
    Extra column could be the raw sentiment
    """
    if isinstance(text, float):
        text = ""

    sentiment_dict = sid_obj.polarity_scores(text)

    compound = sentiment_dict['compound']
    if compound >= 0.05:
        return 1
    elif compound <= -0.05:
        return -1
    else:
        return 0

# Instantiate outside of for loop because of amount of time it requires to repeatedly call
sid_obj = SentimentIntensityAnalyzer()

def encode_adverttitle():
    df['advert_title_sentiment'] = [get_sentiment(x, sid_obj) for x in df.adverttitle]

def encode_advert():
    df['advert_sentiment'] = [get_sentiment(x, sid_obj) for x in df.advert]


# Could also add extra cols like advert title length / advert length (in chars)

In [20]:
# df[['advert_title_sentiment', 'advert_sentiment']].to_csv('tmp_sentiment_file.csv', index=False)

In [21]:
group_owners()
add_service_history_flag()
add_manufactured_year()
add_private_plate_flag()
add_convertible_flag()
add_imported_flag()
add_trim_flag()
add_ulez_flag()
add_vrm_flag()
add_website_flag()
calculate_vehicle_age()
add_mileage_deviation()
group_care_makes()
add_manual_flag()
group_fuel_types()
group_doors()
group_seats()
mileage_to_integer()
group_engine_sizes()
add_new_flag()
group_co2()
count_images()
add_postcode_flag()
extract_lat_long_from_postcode()

# Below take several mins so have extracted to file
# encode_adverttitle()
# encode_advert()
sentiment_df = pd.read_csv("tmp_sentiment_file.csv")
df['advert_sentiment'] = sentiment_df['advert_sentiment']
df['advert_title_sentiment'] = sentiment_df['advert_title_sentiment']

# df = encode_with_hashing(df, 'model', n_components=16)


In [22]:
cols_to_drop = ['vrm', 'imported', 'trim', 'body_type', 'todaysdate', 'sellerwebsite', 
'service_history', 'manufactured_year', 'emission_scheme', 'mileageDeviation', 'make', 'transmission', 'model', 'fuel_type', 'generation',
'doors', 'seats', 'owners', 'engine_size', 'condition', 'year', 'co2Emissions', 'images', 'sellerpostcode', 'annual_tax', 
'adverttitle', 'advert', 'year_of_manufacture'
]
for col in cols_to_drop:
    try:
        df.drop(columns=col, inplace=True)
    except Exception:
        continue
df.columns

Index(['mileage', 'price', 'owners_grouped', 'has_service_history',
       'is_private_plate', 'is_convertible', 'is_imported', 'has_trim',
       'is_ulez', 'known_reg_plate', 'has_website', 'vehicle_age',
       'mileage_deviation_encoded', 'make_grouped', 'is_manual',
       'fuel_type_grouped', 'doors_grouped', 'seats_grouped',
       'engine_size_grouped', 'is_new', 'co2_grouped', 'image_count',
       'has_postcode', 'long_lat_feature', 'advert_sentiment',
       'advert_title_sentiment'],
      dtype='object')

# Columns that require splitting before encoding
These columns will most likely be ordinally encoded on the median price. To prevent data leakage, we have to split the data before deriving the encoding.

1. CO2 Emissions
2. Annual tax
3. Make
4. doors
5. seats
6. engine size
7. Fuel Type

In [23]:
print(df.columns)
df.head()

Index(['mileage', 'price', 'owners_grouped', 'has_service_history',
       'is_private_plate', 'is_convertible', 'is_imported', 'has_trim',
       'is_ulez', 'known_reg_plate', 'has_website', 'vehicle_age',
       'mileage_deviation_encoded', 'make_grouped', 'is_manual',
       'fuel_type_grouped', 'doors_grouped', 'seats_grouped',
       'engine_size_grouped', 'is_new', 'co2_grouped', 'image_count',
       'has_postcode', 'long_lat_feature', 'advert_sentiment',
       'advert_title_sentiment'],
      dtype='object')


,mileage,price,owners_grouped,has_service_history,is_private_plate,is_convertible,is_imported,has_trim,is_ulez,known_reg_plate,...,doors_grouped,seats_grouped,engine_size_grouped,is_new,co2_grouped,image_count,has_postcode,long_lat_feature,advert_sentiment,advert_title_sentiment
0,19000.0,17995,unknown,1,0,0,1,1,0,1,...,4 doors,5 seats,large,0,band13,23,1,5828.832168,1,0
1,17464.0,16995,unknown,1,0,0,0,1,1,1,...,3 doors,4 seats,medium,0,band8,12,1,5730.618520,1,0
2,8500.0,22995,2to4 owners,1,0,0,0,1,1,1,...,3 doors,2 seats,medium,0,band8,15,1,5891.412663,1,0
3,17000.0,11995,unknown,1,0,1,0,1,1,1,...,2 doors,4 seats,medium,0,band13,16,1,5730.618520,1,0
4,12900.0,11990,2to4 owners,1,0,1,0,1,1,1,...,2 doors,4 seats,medium,0,band8,20,1,5886.345407,1,0


In [24]:
cols = ['make_grouped', 'doors_grouped', 'seats_grouped', 'engine_size_grouped', 'co2_grouped', 'fuel_type_grouped', 'mileage']

df[cols].head()

,make_grouped,doors_grouped,seats_grouped,engine_size_grouped,co2_grouped,fuel_type_grouped,mileage
0,rare,4 doors,5 seats,large,band13,petrol,19000.0
1,rare,3 doors,4 seats,medium,band8,petrol,17464.0
2,rare,3 doors,2 seats,medium,band8,petrol,8500.0
3,rare,2 doors,4 seats,medium,band13,petrol,17000.0
4,rare,2 doors,4 seats,medium,band8,petrol,12900.0


In [47]:
def get_ordinal(x, x_train, y_train):
    X_train_copy = x_train.copy()
    X_train_copy['price'] = y_train
    med_dict = X_train_copy.groupby(x)['price'].median()
    med_dict_sorted = {k: v for k, v in sorted(med_dict.items(), key=lambda item: item[1])}
    label = 0
    for k, v in med_dict_sorted.items():
        med_dict_sorted[k] = label
        label += 1
    return med_dict_sorted

def map_to_ordinal(col, mapping, df=df):
    df[col] = df[col].map(mapping)


### Split the data

In [26]:
X_train, X_test, y_train, y_test = DataLoader.split_data(df)

In [46]:
y_train.head()

56242     26490
75824    137000
44564      5700
33867      4690
62544      3995
Name: price, dtype: int64

In [30]:
def encode_mileage():
    # Encode Train
    X_train['mileage_quartile'] = [group_on_quartile(x, 'mileage', df=X_train) for x in X_train['mileage']]
    # Encode Test using Train
    X_test['mileage_quartile'] = [group_on_quartile(x, 'mileage', df=X_train) for x in X_test['mileage']]
    X_train.drop(columns=['mileage'], inplace=True)
    X_test.drop(columns=['mileage'], inplace=True)

encode_mileage()

In [51]:
owners_mapping = get_ordinal('owners_grouped', X_train, y_train)
map_to_ordinal('owners_grouped', owners_mapping, df=X_train)
map_to_ordinal('owners_grouped', owners_mapping, df=X_test)
X_train.drop(columns=['owners_grouped'], inplace=True)
X_test.drop(columns=['owners_grouped'], inplace=True)

In [53]:
"""
1. CO2 Emissions
2. Annual tax
3. Make
4. doors
5. seats
6. engine size
7. Fuel Type
"""

def to_ordinal(col):
    col_mapping = get_ordinal(col, X_train, y_train)
    map_to_ordinal(col, col_mapping, df=X_train)
    map_to_ordinal(col, col_mapping, df=X_test)
    X_train.drop(columns=[col], inplace=True)
    X_test.drop(columns=[col], inplace=True)

In [54]:
to_ordinal('co2_grouped')
to_ordinal('make_grouped')
to_ordinal('doors_grouped')
to_ordinal('seats_grouped')
to_ordinal('engine_size_grouped')
to_ordinal('fuel_type_grouped')

In [59]:
X_train.dtypes

has_service_history            int64
is_private_plate               int64
is_convertible                 int64
is_imported                    int64
has_trim                       int64
is_ulez                        int64
known_reg_plate                int64
has_website                    int64
vehicle_age                  float64
mileage_deviation_encoded      int64
is_manual                      int64
is_new                         int64
image_count                    int64
has_postcode                   int64
long_lat_feature             float64
advert_sentiment               int64
advert_title_sentiment         int64
mileage_quartile               int64
dtype: object